# Анализ метрик на датасете Wikidata Big (Temporal Knowledge Graph)

Этот блокнот демонстрирует процесс адаптации существующей системы к работе с крупномасштабным темпоральным графом знаний **Wikidata Big**. 

**Основные шаги:**
1. Загрузка данных Wikidata (сущности, отношения, временные метки).
2. Отображение (Mapping) числовых ID в строковые представления Wikidata (Q-IDs, P-IDs).
3. Инициализация векторной модели и индексация темпоральных триплетов.
4. Расчет метрик качества (Hits@K, MRR) с учетом временной координаты.

In [1]:
# Конфигурация: использовать ТОЛЬКО PyTorch (не TensorFlow)
import os
os.environ['USE_TF'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

print('✓ Настроен PyTorch-only режим')

✓ Настроен PyTorch-only режим


In [2]:
import sys
import os
import pickle
import numpy as np
import random
import shutil
from tqdm.auto import tqdm
import torch

# Добавление src в путь для импорта модулей проекта
sys.path.append('../../')

# Импорт необходимых компонентов системы
from src.kg_model.embeddings_model import EmbeddingsModel, EmbeddingsModelConfig, EmbedderModelConfig
from src.db_drivers.vector_driver import VectorDriverConfig, VectorDBConnectionConfig, VectorDBInstance
from src.utils.data_structs import TripletCreator, NodeCreator, NodeType, RelationCreator, RelationType
from src.utils import Logger

/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at schema.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at common.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at milvus.proto. Please update the gencode to avoid compatibility violations in the next runtime 

## 1. Загрузка и подготовка данных Wikidata

Данные WikidataBig хранятся в формате pickle и содержат:
- `ent_id`: словарь соответствия Q-идентификаторов (строк) числовым ID.
- `rel_id`: словарь соответствия P-идентификаторов (отношений) числовым ID.
- `ts_id`: словарь временных меток.
- `test.pickle` / `valid.pickle`: массивы триплетов с временными интервалами.

In [3]:
DATA_PATH = '../../wikidata_big/kg/tkbc_processed_data/wikidata_big/'

def load_wikidata_mapping(file_name):
    with open(os.path.join(DATA_PATH, file_name), 'rb') as f:
        return pickle.load(f)

print("Загрузка словарей соответствия...")
ent_to_id = load_wikidata_mapping('ent_id')
rel_to_id = load_wikidata_mapping('rel_id')
ts_to_id = load_wikidata_mapping('ts_id')

# Создаем обратные словари для восстановления строк по ID
id_to_ent = {v: k for k, v in ent_to_id.items()}
id_to_rel = {v: k for k, v in rel_to_id.items()}
id_to_ts = {v: str(k) for k, v in ts_to_id.items()}

print(f"Загружено сущностей: {len(ent_to_id)}")
print(f"Загружено отношений: {len(rel_to_id)}")
print(f"Загружено временных меток: {len(ts_to_id)}")

Загрузка словарей соответствия...
Загружено сущностей: 125726
Загружено отношений: 203
Загружено временных меток: 9621


In [4]:
print("Загрузка тестовых данных...")
with open(os.path.join(DATA_PATH, 'test.pickle'), 'rb') as f:
    test_data = pickle.load(f) # Формат: (s, r, o, start_t, end_t)

print(f"Количество тестовых квадруплетов: {len(test_data)}")

Загрузка тестовых данных...
Количество тестовых квадруплетов: 4995


## 2. Конвертация в формат системы (Triplet)

Мы преобразуем числовые данные Wikidata в объекты `Triplet`, используя нашу обновленную структуру с поддержкой времени.

In [5]:
def convert_to_triplets(data_subset, sample_limit=4995):
    # Берем подмножество для ускорения демонстрации
    subset = data_subset[:sample_limit]
    converted = []
    
    for row in tqdm(subset, desc="Конвертация"):
        # Извлекаем названия из ID
        s_name = id_to_ent[row[0]]
        r_name = id_to_rel[row[1]]
        o_name = id_to_ent[row[2]]
        t_name = id_to_ts[row[3]] # Используем стартовое время как основной маркер
        
        # Создаем узлы и отношение
        s_node = NodeCreator.create(NodeType.object, s_name)
        r_rel = RelationCreator.create(RelationType.simple, r_name)
        o_node = NodeCreator.create(NodeType.object, o_name)
        t_node = NodeCreator.create(NodeType.time, t_name)
        
        # Создаем темпоральный триплет (S, P, O, T)
        triplet = TripletCreator.create(s_node, r_rel, o_node, time=t_node)
        converted.append(triplet)
        
    return converted

test_triplets = convert_to_triplets(test_data)
print(f"Пример: {test_triplets[0].stringified}")

Конвертация:   0%|          | 0/4995 [00:00<?, ?it/s]

Пример: (1918, 0, 0): Q762215 P241 Q9212


## 3. Инициализация модели и расчет метрик

Мы инициализируем `EmbeddingsModel` и проводим поиск по векторизованному представлению запроса `(s, p, t)` для нахождения верного `o`.

In [6]:
# Конфигурация векторного хранилища (тестовая)
NODES_DB_PATH = '../../data/graph_structures/vectorized_nodes/wikidata_test'
TRIPLETS_DB_PATH = '../../data/graph_structures/vectorized_triplets/wikidata_test'
# Используем HuggingFace Hub если локальная модель отсутствует
EMBEDDER_PATH = '../../models/wikidata_finetuned'
if not os.path.exists(EMBEDDER_PATH):
    EMBEDDER_PATH = 'intfloat/multilingual-e5-small'
# Автоопределение устройства (CUDA/MPS/CPU)
from src.utils.device_utils import get_device
DEVICE = get_device()

# Очистка предыдущих тестов
for path in [NODES_DB_PATH, TRIPLETS_DB_PATH]:
    if os.path.exists(path): shutil.rmtree(path)

config = EmbeddingsModelConfig(
    nodesdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(conn={"path": NODES_DB_PATH}, need_to_clear=True)),
    tripletsdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(conn={"path": TRIPLETS_DB_PATH}, need_to_clear=True)),
    embedder_config=EmbedderModelConfig(model_name_or_path=EMBEDDER_PATH, device=DEVICE)
)

model = EmbeddingsModel(config)
# model.embedder.init_model()

# Индексация данных
print("Индексация тестовых триплетов в векторную БД...")
model.create_triplets(test_triplets)

✓ Используется Apple Silicon GPU (MPS)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

Индексация тестовых триплетов в векторную БД...


100%|██████████| 40/40 [00:23<00:00,  1.68it/s]


{'nodes': {'0588318a2d8b52eeaab0f2db54ffbf15',
  '82fabf9c00a1de0b9c44a58939582ce5',
  '870418317edfffda060de297ee855a25',
  '4484ae2d65c5fa3fb009c46edd0b527d',
  '6b599aea53b87f33d2c067c95fd4a9d6',
  'a1147558a58df7384023581be924e57a',
  'f32816d4d4b725e1e18349a1b7b8bcdf',
  '6855bb85352895c7d3d56f3c70254bcc',
  'fe36add5b58e0413d99600e004a68de6',
  'e6bd8c15f648e4359edd8f5e3a745116',
  'ed9937c7acc8597508331c9cf69cbc02',
  '96e4b7066c30f136dd07b1e3dd3277bd',
  'e002fa0ca171baddf490712bcc138323',
  '089ac28fcca84e36053255cdd697694f',
  '4c61414e2cdae9f493d6a51081f97213',
  'c1dddcaac8d0ed5cde49dd58844d5e28',
  'd62d7d8f226e55ffe8bd459858b8a929',
  '57820a3ed7c059210512b2bd29ffc185',
  '26fea7e9bc743592f25c78bd4b5cec96',
  '7ceaf968b29d5a1d19e885aaeb06e349',
  '631f661820e62ea6e3ea39e6c4b2949c',
  'bd8808c1013e728169dcae91146f1427',
  '1b1dcd67324cee8fc35a1ca9a91bc9d8',
  '35d37d0afb8fdf1234d274e3726ca35d',
  '9282e51a56bb338602fc239be7388f8f',
  '27a9e40f53d9dba6fdae110f2a32b8ed',
  '

## 4. Расчет MRR и Hits@K

**Метрики:**
- **Hits@K**: Доля случаев, когда правильный ответ находится в топ-K результатах.
- **MRR (Mean Reciprocal Rank)**: Среднее значение величины, обратной рангу правильного ответа.

In [7]:

def evaluate_comprehensive(model, triplets, k_values=[1, 5, 10, 100]):
    metrics = {
        'object': {'hits': {k: 0 for k in k_values}, 'mrr': 0, 'count': 0},
        'subject': {'hits': {k: 0 for k in k_values}, 'mrr': 0, 'count': 0},
        'time': {'hits': {k: 0 for k in k_values}, 'mrr': 0, 'count': 0}
    }

    print(f"Starting comprehensive evaluation on {len(triplets)} triplets...")

    for triplet in tqdm(triplets, desc="Оценка"):
        t_str = triplet.time.name if triplet.time else ""
        s_str = triplet.start_node.name
        r_str = triplet.relation.name
        o_str = triplet.end_node.name
        
        # 1. Predict Object: (S, R, T) -> O
        # Query: "Time: {t} | Subject Relation"
        query_o = f"{t_str}: {s_str} {r_str}"
        _update_metrics(model, query_o, triplet.end_node.id, metrics['object'], k_values)
        
        # 2. Predict Subject: (O, R, T) -> S
        # Query usage: "Time: {t} | Relation Object" (Trying to predict Subject)
        # Note: This is a naive attempt to see if the model can infer subject.
        query_s = f"{t_str}: {r_str} {o_str}"
        _update_metrics(model, query_s, triplet.start_node.id, metrics['subject'], k_values)
        
        # 3. Predict Time: (S, R, O) -> T
        if triplet.time:
            # Query usage: "Subject Relation Object" (Predicting Time)
            query_t = f"{s_str} {r_str} {o_str}"
            _update_metrics(model, query_t, triplet.time.id, metrics['time'], k_values)

    # Normalize
    results = {}
    for m_type, data in metrics.items():
        count = data['count']
        if count > 0:
            res = {'mrr': data['mrr'] / count}
            res.update({f'hits@{k}': data['hits'][k] / count for k in k_values})
            results[m_type] = res
        else:
            results[m_type] = None
            
    return results

def _update_metrics(model, query_text, gold_id, metric_dict, k_values):
    # Retrieve
    # Note: passing instructions/prompts if needed. Here we assume naive usage.
    # The embedder might need a "query: " prefix depending on config.
    # E5 expects "query: " for queries. checks EmbeddingsModel config.
    # But here we just call encode_passages. 
    # EmbedderModel.encode_passages expects list of strings.
    
    # We should filter 'ids' from include to avoid Chroma errors
    try:
        q_emb = model.embedder.encode_passages([query_text])[0]
        query_inst = VectorDBInstance(id='q', document=query_text, embedding=q_emb)
        
        results = model.vectordbs['nodes'].retrieve([query_inst], n_results=100, includes=[])
        candidates = results[0]
        
        rank = None
        for i, (dist, inst) in enumerate(candidates):
            if inst.id == gold_id:
                rank = i + 1
                break
                
        if rank is not None:
            metric_dict['mrr'] += 1.0 / rank
            for k in k_values:
                if rank <= k: metric_dict['hits'][k] += 1
                
        metric_dict['count'] += 1
    except Exception as e:
        # print(f"Error in metric update: {e}")
        pass


In [8]:

results = evaluate_comprehensive(model, test_triplets)

print("\n=== РЕЗУЛЬТАТЫ WIKIDATA BIG ===")
for task, res in results.items():
    if res:
        print(f"\n[{task.upper()} PREDICTION]")
        print(f"MRR: {res['mrr']}")
        for k in [1, 5, 10, 100]:
            print(f"Hits@{k}: {res[f'hits@{k}']}")
    else:
        print(f"\n[{task.upper()}] No data")


Starting comprehensive evaluation on 4995 triplets...


Оценка:   0%|          | 0/4995 [00:00<?, ?it/s]


=== РЕЗУЛЬТАТЫ WIKIDATA BIG ===

[OBJECT PREDICTION]
MRR: 0.017368480829200925
Hits@1: 0.0032032032032032033
Hits@5: 0.02122122122122122
Hits@10: 0.03423423423423423
Hits@100: 0.22662662662662664

[SUBJECT PREDICTION]
MRR: 0.00036514665345721196
Hits@1: 0.0
Hits@5: 0.0004004004004004004
Hits@10: 0.0004004004004004004
Hits@100: 0.01021021021021021

[TIME PREDICTION]
MRR: 0.0
Hits@1: 0.0
Hits@5: 0.0
Hits@10: 0.0
Hits@100: 0.0



### Анализ результатов и объяснение метрик

В данном эксперименте мы наблюдаем невысокие значения метрик (MRR, Hits@1/5/10) для всех типов задач (предсказание объекта, субъекта и времени). Это **ожидаемый результат** на данном этапе разработки, обусловленный следующими факторами:

1.  **Наивный подход (Zero-Shot)**:
    *   Мы используем **предобученную E5-модель** в режиме "из коробки" (zero-shot). Модель обучалась на больших текстовых корпусах (MS-MARCO, C4) для задач информационного поиска (Information Retrieval), а не семантической близости в графах знаний.
    *   Модель никогда не видела наши специфические данные (Wikidata entities) и не дообучалась (fine-tuning) на них.

2.  **Структура запросов**:
    *   Мы формируем запросы путем простой конкатенации строк: `2020: Piter Pan works_as`. Это неестественная языковая конструкция. Языковые модели намного лучше работают с естественными вопросами ("Who works as Piter Pan in 2020?").
    *   Отсутствие специальных токенов или разделителей, которые модель могла бы выучить, затрудняет понимание роли каждого слова.

3.  **Неоднозначность сущностей**:
    *   Векторная база данных содержит эмбеддинги *названий* сущностей (например, "Paris"). Без контекста слово "Paris" может означать город, имя человека или мифического персонажа. При поиске ближайших соседей модель возвращает все похожие по смыслу слова, что снижает точность (Rank) правильного ответа.

4.  **Разделение времени**:
    *   Предсказание времени (Time Prediction) особенно сложно, так как "2020" и "2021" лексически очень близки, и General Purpose embedding models часто не различают числа и даты так тонко, как специализированные TKG (Temporal Knowledge Graph) модели.

### Пути улучшения (Next Steps)

Чтобы значительно повысить качество метрик, необходимо внедрить следующие улучшения:

1.  **Fine-tuning (Дообучение)**:
    *   Дообучить E5 или BERT-подобную модель на парах `(Query, Positive_Entity)` из нашей обучающей выборки. Это самый сильный рычаг для роста качества.

2.  **Архитектурные улучшения**:
    *   Использовать **Graph Neural Networks (GNN)**, такие как RGCN или T-GCN, которые учитывают не только текст, но и связи в графе.
    *   Применить **Knowledge Graph Embeddings (KGE)** методы: TransE, ComplEx, TComplEx.

3.  **Улучшение генерации запросов**:
    *   Переписать генерацию запросов в естественный язык с помощью LLM (например, "At what time did X relation Y?").
